Libraries

In [ ]:
import random 

import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from skimage.feature import hog

from matplotlib import pyplot as plt

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.listdir('/content/drive/MyDrive/ckplus')

['happy', 'contempt', 'disgust', 'surprise', 'fear', 'sadness', 'anger']

In [ ]:
data_path = '/content/drive/MyDrive/ckplus'
data_dir_list = os.listdir(data_path)

img_data_list=[]


for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(48,48))
        img_data_list.append(input_img_resize)
        
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

Loaded the images of dataset-happy

Loaded the images of dataset-contempt

Loaded the images of dataset-disgust

Loaded the images of dataset-surprise

Loaded the images of dataset-fear

Loaded the images of dataset-sadness

Loaded the images of dataset-anger



(981, 48, 48, 3)

In [ ]:
len(img_data)

981

Read the Dataset



In [ ]:
# import the necessary packages
import os

image_types = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")


def list_images(basePath, contains=None):
    return list_files(basePath, validExts=image_types, contains=contains)


def list_files(basePath, validExts=None, contains=None):
    # loop over the directory structure
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue

            # determine the file extension of the current file
            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if validExts is None or ext.endswith(validExts):
                # construct the path to the image and yield it
                imagePath = os.path.join(rootDir, filename)
                yield imagePath

imagePaths = list(list_images('/content/drive/MyDrive/ckplus'))


Read and convert images into gray-scale



In [ ]:
def colortogray(im):
    image = cv2.imread(im)
    imgray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return imgray

In [ ]:
DATADIR='/content/drive/MyDrive/ckplus'
CATEGORIES = os.listdir(DATADIR)


Resize images with a fixed size(INTER LINEAR interpolation)



In [ ]:
def resizeImage(image, size):
    return cv2.resize(image, (size,size))

Features extraction with HOG algorithm



In [ ]:
def feat_lab(imagePaths):

    features = []
    labels = []

    for imagePath in imagePaths:
        im = colortogray(imagePath)
        im = resizeImage(im,64)
        fd1 =  hog(im, orientations=7, pixels_per_cell=(8, 8),cells_per_block=(4, 4),block_norm= 'L2-Hys' ,transform_sqrt = False)

        label = imagePath.split(os.path.sep)[-2]
        labels.append(label)
        features.append(fd1)

    features = np.array(features)
    labels = np.array(labels)
    return features,labels

Compute the features and obtain the lables using the previous functions



In [ ]:
features,labels = feat_lab(imagePaths)


print("[INFO] The number of features ..." + str(features.shape[1]))

print("[INFO] The number of samples ..." + str(labels.shape[0]))

[INFO] The number of features ...2800
[INFO] The number of samples ...981


The classifier

In [ ]:
svm_clf = SVC(kernel='rbf', gamma='scale', C= 10)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = classification_report(y_train, pred)
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = classification_report(y_test, pred)
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

################################################################################
################################################################################


R =random.randint(1,88)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3,random_state=40
                                                   )

print("[INFO] The number of images used in training ..." + str(X_train.shape[0]))
print("[INFO] The number of images used in testing ..." + str(X_test.shape[0]))


svm_clf.fit(X_train, y_train)

print_score(svm_clf, X_train, y_train, X_test, y_test, train=True)
print_score(svm_clf, X_train, y_train, X_test, y_test, train=False)


[INFO] The number of images used in training ...686
[INFO] The number of images used in testing ...295
Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00        90
    contempt       1.00      1.00      1.00        39
     disgust       1.00      1.00      1.00       129
        fear       1.00      1.00      1.00        52
       happy       1.00      1.00      1.00       144
     sadness       1.00      1.00      1.00        57
    surprise       1.00      1.00      1.00       175

    accuracy                           1.00       686
   macro avg       1.00      1.00      1.00       686
weighted avg       1.00      1.00      1.00       686

_______________________________________________
Confusion Matrix: 
 [[ 90   0   0   0   0   0   0]
 [  0  39   0   0   0   0   0]
 [  0   0 129   0   0   0   0]
 [  0   0   0  52   0   0   0]

**Cross-validation accuracy**

*3-FOLDS*

In [ ]:
cv = KFold(n_splits=3, random_state=1, shuffle=True)
# evaluate model
scores = cross_val_score(svm_clf, features, labels, scoring='accuracy', cv=cv, n_jobs=-1)
# Report performance

print('SVM MEAN  Accuracy: ',str(np.mean(scores)*100)[:5] + '%')
print('Standard deviation: ',str(np.std(scores)*100)[:5] + '%')

SVM MEAN  Accuracy:  99.18%
Standard deviation:  0.144%


*5-FOLDS*

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)
# evaluate model
scores = cross_val_score(svm_clf, features, labels, scoring='accuracy', cv=cv, n_jobs=-1)
# Report performance

print('SVM MEAN  Accuracy: ',str(np.mean(scores)*100)[:5] + '%')
print('Standard deviation: ',str(np.std(scores)*100)[:5] + '%')

SVM MEAN  Accuracy:  98.97%
Standard deviation:  0.968%


10-FOLDS

In [ ]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# evaluate model
scores = cross_val_score(svm_clf, features, labels, scoring='accuracy', cv=cv, n_jobs=-1)
# Report performance

print('SVM MEAN  Accuracy: ',str(np.mean(scores)*100)[:5] + '%')
print('Standard deviation: ',str(np.std(scores)*100)[:5] + '%')

SVM MEAN  Accuracy:  99.69%
Standard deviation:  0.918%


In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from itertools import product
from sklearn.ensemble import VotingClassifier

X = features
y = labels

R =random.randint(1,88)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3,random_state=40)

print("[INFO] The number of images used in training ..." + str(X_train.shape[0]))
print("[INFO] The number of images used in testing ..." + str(X_test.shape[0]))



 # Training classifiers
#clf1 = DecisionTreeClassifier(max_depth=4)
clf1 = KNeighborsClassifier(n_neighbors=7)
clf3 = SVC(kernel='rbf', probability=True)
eclf = VotingClassifier(estimators=[('dt', clf1), ('knn', clf2), ('svc', clf3)], voting='soft', weights=[2, 1, 2])

clf1 = clf1.fit(X_train, y_train)
#clf2 = clf2.fit(X_train, y_train)
clf3 = clf3.fit(X_train, y_train)
eclf = eclf.fit(X_train, y_train)



[INFO] The number of images used in training ...686
[INFO] The number of images used in testing ...295


In [ ]:
y_pred = eclf.predict(X_test)

In [ ]:
print_score(eclf, X_train, y_train, X_test, y_test, train=True)

Train Result:
Accuracy Score: 99.56%
_______________________________________________
CLASSIFICATION REPORT:
              precision    recall  f1-score   support

       anger       0.99      0.98      0.98        90
    contempt       1.00      1.00      1.00        39
     disgust       1.00      1.00      1.00       129
        fear       1.00      1.00      1.00        52
       happy       1.00      1.00      1.00       144
     sadness       0.97      0.98      0.97        57
    surprise       1.00      1.00      1.00       175

    accuracy                           1.00       686
   macro avg       0.99      0.99      0.99       686
weighted avg       1.00      1.00      1.00       686

_______________________________________________
Confusion Matrix: 
 [[ 88   0   0   0   0   2   0]
 [  0  39   0   0   0   0   0]
 [  0   0 129   0   0   0   0]
 [  0   0   0  52   0   0   0]
 [  0   0   0   0 144   0   0]
 [  1   0   0   0   0  56   0]
 [  0   0   0   0   0   0 175]]



In [ ]:
y_pred[:7]

array(['disgust', 'contempt', 'surprise', 'happy', 'happy', 'disgust',
       'anger'], dtype='<U8')

In [ ]:
print_score(clf1, X_train, y_train, X_test, y_test, train=True)

Train Result:
Accuracy Score: 80.32%
_______________________________________________
CLASSIFICATION REPORT:
              precision    recall  f1-score   support

       anger       0.66      0.79      0.72        90
    contempt       0.76      0.82      0.79        39
     disgust       0.87      0.82      0.84       129
        fear       0.56      0.44      0.49        52
       happy       0.83      0.89      0.86       144
     sadness       0.66      0.51      0.57        57
    surprise       0.92      0.93      0.92       175

    accuracy                           0.80       686
   macro avg       0.75      0.74      0.74       686
weighted avg       0.80      0.80      0.80       686

_______________________________________________
Confusion Matrix: 
 [[ 71   1   5   6   3   3   1]
 [  3  32   0   0   1   3   0]
 [  5   0 106   2  11   3   2]
 [  5   5   4  23   4   2   9]
 [  6   2   5   3 128   0   0]
 [ 13   2   2   5   4  29   2]
 [  4   0   0   2   3   4 162]]



In [ ]:
print_score(clf3, X_train, y_train, X_test, y_test, train=True)

Train Result:
Accuracy Score: 99.27%
_______________________________________________
CLASSIFICATION REPORT:
              precision    recall  f1-score   support

       anger       0.99      0.98      0.98        90
    contempt       1.00      0.97      0.99        39
     disgust       1.00      1.00      1.00       129
        fear       1.00      0.98      0.99        52
       happy       0.99      1.00      0.99       144
     sadness       0.97      0.98      0.97        57
    surprise       1.00      1.00      1.00       175

    accuracy                           0.99       686
   macro avg       0.99      0.99      0.99       686
weighted avg       0.99      0.99      0.99       686

_______________________________________________
Confusion Matrix: 
 [[ 88   0   0   0   0   2   0]
 [  0  38   0   0   1   0   0]
 [  0   0 129   0   0   0   0]
 [  0   0   0  51   1   0   0]
 [  0   0   0   0 144   0   0]
 [  1   0   0   0   0  56   0]
 [  0   0   0   0   0   0 175]]

